In [28]:
import sys
import pandas as pd
from  random import randint
sys.path.append("..")
from next_word_prediction import GPT2
from sklearn.metrics import accuracy_score
from IPython.display import clear_output
clear_output(wait=True)

In [29]:
gpt2 = GPT2()

In [30]:
data_base = pd.read_csv('../swiss-test/data/COCO-locations-test-20.csv')

In [31]:
after_sentences = [ ' All this happens in the ', ' It was great time in the ', ' Hope to see again this ', ' This place is located in the ']

In [32]:
un_X = data_base['cap']
un_y = data_base['location']
un_y = list(map(lambda array_str: array_str[1:-1].split(', '), un_y))
un_y = list(map(lambda array: list(map(lambda str: str[1:-1], array)), un_y))

X_base = []
y_base = []

for text, label_array in list(zip(un_X, un_y)):
    for label in label_array:
        X_base.append(text)
        y_base.append(label)

X_base = list(map(lambda text: text + after_sentences[randint(0, len(after_sentences) - 1)], X_base))

In [33]:
def get_random_sentences(sentences, n=3):
    result = []
    for _ in range(n):
        result.append(sentences[randint(0, len(sentences) - 1)])
    return result

def predict(y_true, y_pred):
    result = []
    for y_t, y_p in list(zip(y_true, y_pred)):
        if y_t in y_p:
            result.append(y_t)
        else:
            result.append(y_p[0])
    return result


def predict_next_words(text, n=10):
    return gpt2.predict_next(text, n)

def predict_words(sentences, n=10):
    return list(map(lambda text: gpt2.predict_next(text, n), sentences))

In [34]:
base_random = get_random_sentences(list(zip(X_base, y_base)))

In [35]:
for (X_i_base, y_i_base) in base_random:
    predicted_words = predict_next_words(X_i_base)
    print('Original text: {}'.format(X_i_base))
    print('True location: {}'.format(y_i_base))
    print('Predicted locations: {}'.format(predicted_words))
    print('Prediction success: {}'.format(y_i_base in predicted_words))
    print('=' * 40)


Original text: Two girls sit by the fence and watch the sun go down over the mountains. This place is located in the 
True location: mountain
Predicted locations: ['middle', 'mountains', 'heart', 'center', 'footh', 'northern', 'southern', 'city', 'desert', 'same']
Prediction success: False
Original text: The moon rose over the city by the sea. Hope to see again this 
True location: sea
Predicted locations: ['time', 'year', 'day', 'evening', 'morning', 'night', 'summer', 'afternoon', 'week', 'spring']
Prediction success: False
Original text: Several young guys run along the beach to swim in the sea. Hope to see again this 
True location: beach
Predicted locations: ['summer', 'year', 'weekend', 'week', 'time', 'spring', 'fall', 'season', 'morning', 'winter']
Prediction success: False


In [36]:
score = accuracy_score(y_true=y_base, y_pred=predict(y_base, predict_words(X_base)))
accuracy = '{}%'.format(int(100. * score))

print('Accuracy for sentences with location: {}'.format(accuracy))

Accuracy for sentences with location: 24%
